In [ ]:
import openpyxl

kommuner = openpyxl.load_workbook(filename = 'kommuner.xlsx')
sheet = kommuner.active

fylker = {
	3: 'Oslo',
	11: 'Rogaland',
	15: 'Møre og Romsdal',
	18: 'Nordland',
	31: 'Østfold',
	32: 'Akershus',
	33: 'Buskerud',
	34: 'Innlandet',
	39: 'Vestfold',
	40: 'Telemark',
	42: 'Agder',
	46: 'Vestland',
	50: 'Trøndelag',
	55: 'Troms',
	56: 'Finmark',
	21: 'Svalbard',
	22: 'Jan Mayen'
}

kommune_til_fylke = {'Abroad (utlandet)': 'Utlandet'}

row_counter = 2
while True:
	if sheet[f'A{row_counter}'].value is None:
		break

	kommune = sheet[f'B{row_counter}'].value
	nummer = sheet[f'A{row_counter}'].value
	nummer = int(str(nummer)[:-2])

	kommune_til_fylke[kommune] = fylker[nummer]
	
	row_counter += 1

In [ ]:
def format_workbook(workbook):
	# Adjust column widths and add cell borders
	border_side = openpyxl.styles.borders.Side(style='thin')
	border_style = openpyxl.styles.borders.Border(left=border_side, top=border_side, right=border_side, bottom=border_side)
	for sheet in workbook.worksheets:
		for column in sheet.columns:
			width = 0
			letter = column[0].column_letter
		
			for cell in column:
				if not isinstance(cell.value, openpyxl.worksheet.formula.ArrayFormula) and not (isinstance(cell.value, str) and cell.value.startswith('=')):
					width = max(width, len(str(cell.value)))
				cell.border = border_style
		
			width += 4
		
			if sheet.title == 'Evalueringer' and letter in reviewer_to_letter.values() or sheet.title == 'Uenighet' and ord(letter) > 68:
				width = 4
		
			sheet.column_dimensions[letter].width = width

In [ ]:
import csv

accepted = []
declined = []

with open('kontakter.csv', 'r', encoding='utf-8') as file:
	file_reader = csv.reader(file, delimiter=',', quotechar='"')
	# Skip headers
	next(file_reader)
	for row in file_reader:
		name = f'{row[1]} {row[2]}'
		email = row[3]
		municipality = row[4]
		county = kommune_til_fylke[municipality]
		school = row[5]
		school_municipality = row[6]
		school_county = kommune_til_fylke[school_municipality]
		parent_one_relationship = row[7]
		parent_one_name = f'{row[8]} {row[9]}'
		parent_one_email = row[10]
		parent_two_relationship = row[11]
		parent_two_name = f'{row[12]} {row[13]}'
		parent_two_email = row[14]

		entry = (
			name,
			email,
			municipality,
			county,
			school,
			school_municipality,
			school_county,
			parent_one_relationship,
			parent_one_name,
			parent_one_email,
			parent_two_relationship,
			parent_two_name,
			parent_two_email
		)

		if row[15] == 'Interview':
			accepted.append(entry)
		else:
			declined.append(entry)

accepted.sort(key=lambda e: e[0].strip().lower())
declined.sort(key=lambda e: e[0].strip().lower())

headers = [
	'Navn',
	'E-postadresse',
	'Kommune',
	'Fylke',
	'Skolenavn',
	'Skolekommune',
	'Skolefylke',
	'Forhold til foresatt 1',
	'Navn foresatt 1',
	'E-postadresse foresatt 1',
	'Forhold til foresatt 2',
	'Navn foresatt 2',
	'E-postadresse foresatt 2'
]

workbook = openpyxl.Workbook()
sheets = []

sheet = workbook.active
sheet.title = 'Invitert'
sheets.append((sheet, accepted))

sheet = workbook.create_sheet('Avslått')
sheets.append((sheet, declined))

for sheet, contacts in sheets:
	for index, header in enumerate(headers):
		letter = chr(65+index)
		sheet[f'{letter}1'].value = header
		sheet[f'{letter}1'].font = openpyxl.styles.Font(bold=True)
	
	for row_number, row in enumerate(contacts):
		row_number += 2
		for column_number, data in enumerate(row):
			letter = chr(65 + column_number)
			sheet[f'{letter}{row_number}'].value = data

format_workbook(workbook)
workbook.save('kontaktliste.xlsx')